In [11]:
from nctotdb import NCDataModel, ZarrWriter

data_model = NCDataModel('./2m_temperature_stream-oper_daily-mean.nc')
data_model.populate()

zarr_writer = ZarrWriter(data_model, './')
zarr_writer.create_zarr()

/home/juliababa/tcc/src/./2m_temperature_stream-oper_daily-mean.zarr


In [37]:
import apache_beam as beam
import numpy as np
import xarray as xr
import weatherbenchX
from weatherbenchX.data_loaders import xarray_loaders
from weatherbenchX.metrics import deterministic
from weatherbenchX.metrics import base as metrics_base
from weatherbenchX import aggregation
from weatherbenchX import weighting
from weatherbenchX import binning
from weatherbenchX import time_chunks
from weatherbenchX import beam_pipeline

dataset = xr.open_zarr('./2m_temperature_stream-oper_daily-mean.zarr')

dataset['time'].load()

dataset

# prediction_path = 'gs://weatherbench2/datasets/hres/2016-2022-0012-64x32_equiangular_conservative.zarr'
target_path = './2m_temperature_stream-oper_daily-mean.zarr'

variables = ['2m_temperature']
target_data_loader = xarray_loaders.TargetsFromXarray(
    path=target_path,
    variables=variables,
)
# prediction_data_loader = xarray_loaders.PredictionsFromXarray(
#     path=prediction_path,
#     variables=variables,
# )

print(dataset.dims)

print(dataset.load())

init_times = np.array(['2023-01-02T00'], dtype='datetime64[ns]')
lead_times = np.array([00], dtype='timedelta64[h]').astype('timedelta64[ns]')   # To silence xr warnings.

target_chunk = target_data_loader.load_chunk(init_times)


print(target_chunk)




FrozenMappingWarningOnValuesAccess({'time': 181, 'latitude': 3, 'longitude': 4})
<xarray.Dataset> Size: 10kB
Dimensions:         (time: 181, latitude: 3, longitude: 4)
Coordinates:
  * latitude        (latitude) float64 24B -15.55 -15.8 -16.05
  * longitude       (longitude) float64 32B -48.26 -48.01 -47.76 -47.51
  * time            (time) datetime64[ns] 1kB NaT 2023-01-02 ... 2023-06-30
Data variables:
    2m_temperature  (time, latitude, longitude) float32 9kB 296.3 ... 292.8
<xarray.Dataset> Size: 112B
Dimensions:         (latitude: 3, longitude: 4, valid_time: 1)
Coordinates:
  * latitude        (latitude) float64 24B -15.55 -15.8 -16.05
  * longitude       (longitude) float64 32B -48.26 -48.01 -47.76 -47.51
  * valid_time      (valid_time) datetime64[ns] 8B 2023-01-02
Data variables:
    2m_temperature  (valid_time, latitude, longitude) float32 48B 296.7 ... 2...
Attributes: (12/32)
    GRIB_NV:                                  0
    GRIB_Nx:                                  4
  

/tmp/ipykernel_23765/377857607.py:14: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  dataset = xr.open_zarr('./2m_temperature_stream-oper_daily-mean.zarr')
/home/juliababa/.local/lib/python3.10/site-packages/weatherbenchX/data_loaders/xarray_loaders.py:105: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store wi